In [1]:
import pandas as pd
import pickle
from tqdm import tqdm

# from openAI import LLM, Sentiment
from roberta import roBERTa, Sentiment

/Users/nathanmalta/miniconda3/envs/twitter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First, load data from pickle files into pandas dataframes

In [2]:
llm = roBERTa()
airlines = [
    ('American Airlines','scraped_data/american.pkl'),
    ('Delta Airlines', 'scraped_data/delta.pkl'),
    ('Spirit Airlines', 'scraped_data/spirit.pkl'),
    ('Southwest Airlines', 'scraped_data/southwest.pkl')
]

In [3]:
def convert_to_dataframe(raw_data):

    #ignore pandas warning
    pd.options.mode.chained_assignment = None
    
    #convert to pandas dataframe, with only the columns we need
    df_raw = pd.DataFrame(raw_data)
    df = df_raw[['time', 'text', 'comments', 'comments_full']]
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')

    #replace comments with just text and commenter name
    df['comments_full'] = df['comments_full'].apply(lambda x: [(comment['commenter_name'], comment['comment_text'].replace('\n', '')) for comment in x])

    return df

In [4]:
data_dict = {}
for airline_name, pickle_file in airlines:
    with open(pickle_file, 'rb') as f:
        raw_data = pickle.load(f)
    data_dict[airline_name] = convert_to_dataframe(raw_data)

In [5]:
data_dict['Spirit Airlines'].head()

text  \
time                                                                     
2023-07-21 16:46:39  Introducing the flying sensation that’s turnin...   
2023-07-20 19:25:39  Ready for a soaring adventure? 🛫\n\nWayman Avi...   
2023-07-14 14:29:18  Where to next? ✈️ Screenshot to find your next...   
2023-07-13 12:08:20  The Spirit Charitable Foundation is proud to c...   
2023-07-12 15:53:22  Want to #FlyFree in 2023? ✈️\n\nEnter to win f...   

                     comments  \
time                            
2023-07-21 16:46:39        74   
2023-07-20 19:25:39        33   
2023-07-14 14:29:18       162   
2023-07-13 12:08:20        45   
2023-07-12 15:53:22        72   

                                                         comments_full  
time                                                                    
2023-07-21 16:46:39  [(Greene Durham, Spirit Airlines is a great co...  
2023-07-20 19:25:39  [(Devon McCann, Speaking of soaring, I used to...  
2023-07-14 14:29:18  [(Spirit Airlines, Daniela Soriano We'd like t...  
2023-07-13 12:08:20         [(Dalibaso Bhakabha, Very, nice, americe)]  
2023-07-12 15:53:22  [(Jodi Nicole, I just love being able to trave...

In [6]:
airline_usernames = ['Delta Air Lines', 'Spirit Airlines', 'American Airlines', 'Southwest Airlines']
def classify_all_comments(post, airline_name):
    '''Classify the sentiment of all comments in a post
    '''
    positive = 0
    negative = 0
    neutral = 0

    comments = post['comments_full']
    for commenter_name, comment in comments:
        if commenter_name in airline_usernames:
            continue #ignore comments from the airline itself

        sentiment = llm.classifySentiment(comment)
        if sentiment == Sentiment.POSITIVE:
            positive += 1
        elif sentiment == Sentiment.NEGATIVE:
            negative += 1
        elif sentiment == Sentiment.NEUTRAL:
            neutral += 1
        
    return positive, negative, neutral

def classify_dataframe(df):
    sentiments = []
    for i in tqdm(range(len(df))):
        post = df.iloc[i]
        positive, negative, neutral = classify_all_comments(post, 'Delta Airlines')
        sentiments.append((positive, negative, neutral))
    return sentiments

In [7]:
# for airline_name, df in data_dict.items():
df = data_dict['Southwest Airlines']
df['sentiments'] = classify_dataframe(df)
df.to_pickle(f'sentiment_data/southwest.pkl')

data_dict['Delta Airlines']

100%|██████████| 400/400 [13:54<00:00,  2.09s/it]


text  \
time                                                                     
2023-07-18 14:30:00                       What words rhyme with Delta?   
2023-07-13 00:00:00  Delta Air Lines\n\nSharing is caring ❤️️\n\nSe...   
2023-07-12 00:00:00  Delta Air Lines\n\nSilver Medallion Member Law...   
2023-07-11 11:58:57  When you’re out there seeing the world, the wo...   
2023-07-10 12:00:18  🔐 Unlock more than fast, free Wi-Fi.\n\nGet ex...   
...                                                                ...   
2020-05-22 13:57:46  Nothing is more important than the health and ...   
2020-05-19 00:00:00  Delta Air Lines\n\nYour safety always comes fi...   
2020-05-15 00:00:00  Delta Air Lines\n\nThe safety of our customers...   
2020-05-15 14:00:04  Learn about the actions we're taking to make s...   
2020-05-14 19:29:59  With a little help from our Delta family, volu...   

                     comments  \
time                            
2023-07-18 14:30:00      2420   
2023-07-13 00:00:00       303   
2023-07-12 00:00:00      8310   
2023-07-11 11:58:57        93   
2023-07-10 12:00:18       210   
...                       ...   
2020-05-22 13:57:46      1043   
2020-05-19 00:00:00      1948   
2020-05-15 00:00:00      1085   
2020-05-15 14:00:04       357   
2020-05-14 19:29:59       531   

                                                         comments_full  
time                                                                    
2023-07-18 14:30:00  [(Emily Carlton Wagoner, Felta nother trip com...  
2023-07-13 00:00:00  [(Tita Yo, We were in Copenhagen last week. Fl...  
2023-07-12 00:00:00  [(Annetta Burke, Brilliant team work), (Jane V...  
2023-07-11 11:58:57  [(Anita Delta Moncur, I Really Love You Delta ...  
2023-07-10 12:00:18  [(Thomas Poster, Delta Airlines ! Thanks for e...  
...                                                                ...  
2020-05-22 13:57:46  [(Dawn Barrack, I had to fly this past week fr...  
2020-05-19 00:00:00  [(Walter Bass, Delta Air Lines my husband and ...  
2020-05-15 00:00:00  [(Pat Kaufman, Dear Delta, I was booked for di...  
2020-05-15 14:00:04  [(Marylee Martin Pope, Safety is great.... but...  
2020-05-14 19:29:59  [(Angie Stewart Garrett, I just wish they woul...  

[400 rows x 3 columns]

In [17]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(3, 1, figsize=(10, 5))

positive = [sentiment[0] for sentiment in df['sentiments']]
negative = [sentiment[1] for sentiment in df['sentiments']]
neutral = [sentiment[2] for sentiment in df['sentiments']]

ax[0].plot(positive)
ax[1].plot(negative)
ax[2].plot(neutral)

ax[0].set_title('Positive')
ax[1].set_title('Negative')
ax[2].set_title('Neutral')
    

Text(0.5, 1.0, 'Neutral')

In [10]:
#save the dataframes
df.to_pickle(f'sentiment_data/spirit.pkl')